1. kaggle (https://www.kaggle.com/datasets)
2. спарсить самостоятельно, но в этом случае вам может понадобиться разметка (если у вас обучение с учителем)

Пишем простой граббер для Telegram чатов на Python https://medium.com/swlh/build-a-telegram-bot-in-go-in-9-minutes-e06ad38acef1  https://proglib.io/p/pishem-prostoy-grabber-dlya-telegram-chatov-na-python-2019-11-06

Spam Classifier in Python from scratch https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

How to deploy ML models with Telegram Bots https://medium.com/analytics-vidhya/how-to-deploy-machine-learning-models-on-android-and-ios-with-telegram-bots-a6fb16922741

In [20]:
import pandas as pd
# show all cell content
pd.set_option('display.max_colwidth', None)
# pd.set_option('max_colwidth', 110)
pd.set_option('display.max_columns', 50)
import numpy as np
import re

#from gensim.test.utils import common_texts
#from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

#from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, \
                            classification_report, precision_recall_curve, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

#import dill
import os

In [85]:
path = '../../data/spam_detection/'

ham_raw = pd.read_json(os.path.join(path, 'ham.json'))
spam_raw = pd.read_json(os.path.join(path, 'spam.json'))

# Data processing

## Обработка неспамных сообщений

In [22]:
ham_raw.tail(3)

,name,type,id,messages
20442,Новое Измайлово,public_supergroup,1077014744,"{'id': 84724, 'type': 'message', 'date': '2021-08-24T21:09:27', 'from': 'Juriy', 'from_id': 'user229200647', 'reply_to_message_id': 84695, 'text': 'Готов на платной основе.)))'}"
20443,Новое Измайлово,public_supergroup,1077014744,"{'id': 84727, 'type': 'message', 'date': '2021-08-24T21:57:16', 'edited': '2021-08-24T21:57:23', 'from': 'Дмитрий', 'from_id': 'user219508084', 'reply_to_message_id': 84718, 'text': 'Если Дёмин Луг 6/5 то это у моря)))'}"
20444,Новое Измайлово,public_supergroup,1077014744,"{'id': 84728, 'type': 'message', 'date': '2021-08-24T22:10:24', 'from': 'Fidel', 'from_id': 'user429545993', 'reply_to_message_id': 84727, 'file': '(File not included. Change data exporting settings to download.)', 'thumbnail': '(File not included. Change data exporting settings to download.)', 'media_type': 'sticker', 'sticker_emoji': '😄', 'width': 512, 'height': 512, 'text': ''}"


In [23]:
ham = pd.json_normalize(ham_raw['messages'])
ham.head(5)

,id,type,date,from,from_id,text,actor,actor_id,action,members,edited,forwarded_from,file,thumbnail,media_type,mime_type,duration_seconds,width,height,reply_to_message_id,photo,contact_information.first_name,contact_information.last_name,contact_information.phone_number,sticker_emoji,contact_vcard,location_information.latitude,location_information.longitude,via_bot,message_id
0,58320,message,2020-07-07T01:21:52,Alexandr,user362421356,Че за возня опять? Возле верного,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58321,service,2020-07-07T01:22:12,NaN,NaN,,Ismail,user589798156,invite_members,[Ismail],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58322,message,2020-07-07T01:22:32,Lia,user587895366,Что то похожее на стрельбу было и сейчас три машины полицейских,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58323,message,2020-07-07T01:22:47,Ismail,user589798156,А кто-то в курсе что за стрельба?,NaN,NaN,NaN,NaN,2020-07-07T01:23:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58324,message,2020-07-07T01:23:32,Maria Gaurikh,user292667992,"Предупредительный был, видимо , от ментов",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# удалям строки сервисных сообщений (около 10-ти)
ham = ham.drop(ham[ham['type']== 'service'].index)

In [25]:
ham.head(5)

,id,type,date,from,from_id,text,actor,actor_id,action,members,edited,forwarded_from,file,thumbnail,media_type,mime_type,duration_seconds,width,height,reply_to_message_id,photo,contact_information.first_name,contact_information.last_name,contact_information.phone_number,sticker_emoji,contact_vcard,location_information.latitude,location_information.longitude,via_bot,message_id
0,58320,message,2020-07-07T01:21:52,Alexandr,user362421356,Че за возня опять? Возле верного,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58322,message,2020-07-07T01:22:32,Lia,user587895366,Что то похожее на стрельбу было и сейчас три машины полицейских,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58323,message,2020-07-07T01:22:47,Ismail,user589798156,А кто-то в курсе что за стрельба?,NaN,NaN,NaN,NaN,2020-07-07T01:23:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58324,message,2020-07-07T01:23:32,Maria Gaurikh,user292667992,"Предупредительный был, видимо , от ментов",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,58326,message,2020-07-07T01:36:05,Andrey Skalny,user473275267,Всё закончилось. Разборки были с поножовщиной,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Список признаков

In [26]:
ham.columns

Index(['id', 'type', 'date', 'from', 'from_id', 'text', 'actor', 'actor_id',
       'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
       'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
       'reply_to_message_id', 'photo', 'contact_information.first_name',
       'contact_information.last_name', 'contact_information.phone_number',
       'sticker_emoji', 'contact_vcard', 'location_information.latitude',
       'location_information.longitude', 'via_bot', 'message_id'],
      dtype='object')

Уникальные значения по некоторым признакам

In [27]:
all_cols = ['id', 'type', 'date', 'from', 'from_id', 'text', 'actor', 'actor_id',
           'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
           'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
           'reply_to_message_id', 'photo', 'contact_information.first_name',
           'contact_information.last_name', 'contact_information.phone_number',
           'sticker_emoji', 'contact_vcard', 'location_information.latitude',
           'location_information.longitude', 'via_bot', 'message_id']

some_cols = ['type', 'actor_id', 'action', 'file', 'thumbnail', 'media_type',
             'sticker_emoji', 'photo', 'mime_type', 'via_bot',
            ]

for col in some_cols:
    print(col, ham[col].unique(), '\n')

type ['message'] 

actor_id [nan] 

action [nan] 

file [nan '(File not included. Change data exporting settings to download.)'] 

thumbnail [nan '(File not included. Change data exporting settings to download.)'] 

media_type [nan 'video_file' 'sticker' 'voice_message' 'animation'] 

sticker_emoji [nan '👍' '🤔' '💩' '🥺' '😂' '🍳' '😄' '😴' '🤑' '🍿' '❗️' '🙄' '😳' '😯' '😭' '😞' '👌'
 '😱' '😌' '🤟' '🤦\u200d♂️' '😤' '😔' '🤨' '😨' '😀' '😆' '😃' '👋' '👏' '😰' '😐' '😉'
 '🤷\u200d♀️' '🇷🇸' '💬' '🤷\u200d♂️' '🤢' '😡' '🤣' '😏' '😣' '😘' '😊' '🙂' '🤦' '😬'
 '⚒' '🙅\u200d♂️' '😮' '😥' '🤩' '☺️'] 

photo [nan '(File not included. Change data exporting settings to download.)'] 

mime_type [nan 'video/mp4' 'image/jpeg' 'audio/ogg'
 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
 'image/png' 'application/pdf' 'image/gif' 'application/zip'
 'application/octet-stream' 'video/quicktime'] 

via_bot [nan '@gif'] 



In [28]:
ham = ham.drop(['id', 'type', 'date', 'from', 'from_id', 'actor', 'actor_id',
                'action', 'members', 'edited', 'forwarded_from', 'file', 'thumbnail',
                'media_type', 'mime_type', 'duration_seconds', 'width', 'height',
                'reply_to_message_id', 'photo', 'contact_information.first_name',
                'contact_information.last_name', 'contact_information.phone_number',
                'sticker_emoji', 'contact_vcard', 'location_information.latitude',
                'location_information.longitude', 'via_bot', 'message_id'], axis=1)
ham

,text
0,Че за возня опять? Возле верного
2,Что то похожее на стрельбу было и сейчас три машины полицейских
3,А кто-то в курсе что за стрельба?
4,"Предупредительный был, видимо , от ментов"
6,Всё закончилось. Разборки были с поножовщиной
...,...
20440,на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔
20441,Ну вот) значит будем скоро гулять по новому парку 👏🏻
20442,Готов на платной основе.)))
20443,Если Дёмин Луг 6/5 то это у моря)))


In [29]:
ham.tail(10)

,text
20435,Пмж
20436,у моря надо брать )
20437,На море только отдыхать пока)
20438,"В сухую погоду и в выходные вполне (только его гадят несознательные отдыхающие). А так чтобы вечером после работы погулять только по дворам, заставленный машинами"
20439,"Главное чтобы было с кем, а где можно думаю и придумать)) а парк может когда-то и обустроят"
20440,на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔
20441,Ну вот) значит будем скоро гулять по новому парку 👏🏻
20442,Готов на платной основе.)))
20443,Если Дёмин Луг 6/5 то это у моря)))
20444,


In [30]:
def clean_msg(msg):
    if isinstance(msg, list):
        list_len = len(msg) - 2
        for pos, elem in enumerate(msg):
            if isinstance(elem, dict) and pos ==  list_len:
                for key, val in elem.items():
                    if key == 'type' and val == 'hashtag':
                        break
                    if key == 'type' and val == 'bold':
                        break
                    if key == 'type' and val == 'mention_name':
                        break
                    if key == 'text':
                        return val
    if isinstance(msg, str):
        return msg

In [31]:
# Извлечение из json текста, , замена '' на Nan, удаление строк (сообщений) без текста
# обновление индекса после удаление строк (индекс теперь по порядку)
ham = ham['text'].apply(clean_msg).replace('', np.nan).dropna().reset_index(drop=True)
ham

0                                                                   Че за возня опять? Возле верного
1                                    Что то похожее на стрельбу было и сейчас три машины полицейских
2                                                                  А кто-то в курсе что за стрельба?
3                                                          Предупредительный был, видимо , от ментов
4                                                      Всё закончилось. Разборки были с поножовщиной
                                                    ...                                             
18042    Главное чтобы было с кем, а где можно думаю и придумать)) а парк может когда-то и обустроят
18043     на доброделе недавно голосование было о благоустройстве. вроде даже наш лесопарк победил 🤔
18044                                           Ну вот) значит будем скоро гулять по новому парку 👏🏻
18045                                                                    Готов на платной о

## Обработка сообщений со спамом

In [49]:
spam_raw.head(5)

,name,type,id,messages
0,Журнал НИ,private_channel,1212843637,"{'id': 1, 'type': 'service', 'date': '2020-07-07T20:25:57', 'actor': 'Журнал НИ', 'actor_id': 'channel1212843637', 'action': 'create_channel', 'title': 'Журнал НИ', 'text': ''}"
1,Журнал НИ,private_channel,1212843637,"{'id': 4, 'type': 'message', 'date': '2020-07-07T20:27:15', 'from': 'Журнал НИ', 'from_id': 'channel1212843637', 'text': 'Канал успешно добавлен в журнал группы'}"
2,Журнал НИ,private_channel,1212843637,"{'id': 5, 'type': 'message', 'date': '2020-07-07T20:31:15', 'from': 'Журнал НИ', 'from_id': 'channel1212843637', 'text': ['🆔 ', {'type': 'hashtag', 'text': '#НовыйПользователь'}, ' ', {'type': 'hashtag', 'text': '#Вернулся'}, ' ', {'type': 'hashtag', 'text': '#newmember'}, ' 1714516c-2aca-441e-a28d-98600220d2a9 Пользователь вернулся в чат, первая регистрация 7 июля 2020 г. 15:54:28 UTC) ', {'type': 'bold', 'text': 'Группа:'}, ' ', {'type': 'code', 'text': 'Новое Измайлово'}, ' [', {'type': 'mention', 'text': '@new_izmailovo'}, '] [', {'type': 'hashtag', 'text': '#chat1001077014744'}, '] ', {'type': 'bold', 'text': 'Инициатор:'}, ' ', {'type': 'code', 'text': 'Пчелка'}, ' [', {'type': 'mention_name', 'text': '518458040', 'user_id': 518458040}, '] [', {'type': 'hashtag', 'text': '#user518458040'}, '] Время: 7 июля 2020 г. 17:31:15 UTC']}"
3,Журнал НИ,private_channel,1212843637,"{'id': 6, 'type': 'message', 'date': '2020-07-07T20:34:59', 'from': 'Журнал НИ', 'from_id': 'channel1212843637', 'text': ['⚙️ ', {'type': 'hashtag', 'text': '#Настройки'}, ' Пользователь изменил настройки группы ', {'type': 'hashtag', 'text': '#cabinet'}, ' 7bacb66d-5767-46bb-818f-893724ff6b66 Код восстановления: ', {'type': 'code', 'text': '108120.52CE26998559FC84117E31F9588521DAjz3e7CGahpz9ZNrFNaXDsJQUBwxrJG4zVAvNuHfadVKKR6906iiF6MgCrih37ZJVhDYwwzzr2eF7nY4VujGypsrvnsLK9o2PUjfYizKSEI9e4udYd4QUS0Hksb9dnCsR'}, ' Измененные настройки: ', {'type': 'italic', 'text': 'Текст приветствия когда человек присоединяется к чату'}, ': ', {'type': 'code', 'text': '[{""pin"":false,""text"":""Добро пожаловать в чат Новое Измайлово!\nОзнакомтесь с правилами telegra.ph/rules-ni-06-22\nГруппа модерируется в автоматическом режиме ботами. Посты не соответствующие правилам удаляются автоматически.\nНажмите кнопку \""Обязуюсь соблюдать!\"" чтобы писать в чат👇🏻👇🏻👇🏻"",""buttons"":[[{""link"":""https://telegra.ph/rules-ni-06-22"",""text"":""♻️Прочитать правила♻️"",""type"":""Link""}]],""send_after"":0,""remove_after"":60,""disable_notify"":false,""disable_link_preview"":true}]'}, ' ', {'type': 'bold', 'text': 'Группа:'}, ' ', {'type': 'code', 'text': 'Новое Измайлово'}, ' [', {'type': 'mention', 'text': '@new_izmailovo'}, '] [', {'type': 'hashtag', 'text': '#chat1001077014744'}, '] ', {'type': 'bold', 'text': 'Инициатор:'}, ' ', {'type': 'code', 'text': 'Sunlighter'}, ' [', {'type': 'mention', 'text': '@sunlighter'}, '] [', {'type': 'hashtag', 'text': '#user1177305831'}, '] Время: 7 июля 2020 г. 17:34:59 UTC']}"
4,Журнал НИ,private_channel,1212843637,"{'id': 7, 'type': 'message', 'date': '2020-07-07T20:35:36', 'from': 'Журнал НИ', 'from_id': 'channel1212843637', 'text': ['🆔 ', {'type': 'hashtag', 'text': '#НовыйПользователь'}, ' ', {'type': 'hashtag', 'text': '#Вернулся'}, ' ', {'type': 'hashtag', 'text': '#newmember'}, ' f3edae57-e3fb-4882-9148-c9a0951d1075 Пользователь вернулся в чат, первая регистрация 7 июля 2020 г. 14:19:01 UTC) ', {'type': 'bold', 'text': 'Группа:'}, ' ', {'type': 'code', 'text': 'Новое Измайлово'}, ' [', {'type': 'mention', 'text': '@new_izmailovo'}, '] [', {'type': 'hashtag', 'text': '#chat1001077014744'}, '] ', {'type': 'bold', 'text': 'Инициатор:'}, ' ', {'type': 'code', 'text': 'Alexander Kalinichenko'}, ' [', {'type': 'mention', 'text': '@alex_kalinichenko'}, '] [', {'type': 'hashtag', 'text': '#user202193293'}, '] Время: 7 июля 2020 г. 17:35:36 UTC']}"


In [50]:
spam = pd.json_normalize(spam_raw['messages'])
spam.tail(5)

,id,type,date,actor,actor_id,action,title,text,from,from_id
3093,3096,message,2021-08-24T13:25:44,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'}, , {'type': 'code', 'text': 'Новое Измайлово'}, [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'}, , {'type': 'code', 'text': 'Tanya'}, [, {'type': 'mention', 'text': '@lunaverrrr'}, ] [, {'type': 'hashtag', 'text': '#user504256170'}, ]\nВремя: 24 августа 2021 г. 10:25:43 UTC]",Журнал НИ,channel1212843637
3094,3097,message,2021-08-24T13:57:26,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'}, , {'type': 'code', 'text': 'Новое Измайлово'}, [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'}, , {'type': 'code', 'text': 'Александр'}, [, {'type': 'mention_name', 'text': '1993449889', 'user_id': 1993449889}, ] [, {'type': 'hashtag', 'text': '#user1993449889'}, ]\nВремя: 24 августа 2021 г. 10:57:26 UTC]",Журнал НИ,channel1212843637
3095,3098,message,2021-08-24T19:12:21,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'}, , {'type': 'code', 'text': 'Новое Измайлово'}, [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'}, , {'type': 'code', 'text': 'Татьяна'}, [, {'type': 'mention', 'text': '@Tatiana_Belinskaya'}, ] [, {'type': 'hashtag', 'text': '#user1452602842'}, ]\nВремя: 24 августа 2021 г. 16:12:21 UTC]",Журнал НИ,channel1212843637
3096,3099,message,2021-08-24T19:12:25,NaN,NaN,NaN,NaN,"[✅ Пользователь прошедший проверку разблокирован\n\n, {'type': 'bold', 'text': 'Группа:'}, , {'type': 'code', 'text': 'Новое Измайлово'}, [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Инициатор:'}, , {'type': 'code', 'text': 'Татьяна'}, [, {'type': 'mention', 'text': '@Tatiana_Belinskaya'}, ] [, {'type': 'hashtag', 'text': '#user1452602842'}, ]\nВремя: 24 августа 2021 г. 16:12:25 UTC]",Журнал НИ,channel1212843637
3097,3100,message,2021-08-24T21:27:48,NaN,NaN,NaN,NaN,"[🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'}, , {'type': 'code', 'text': 'Новое Измайлово'}, [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'}, , {'type': 'code', 'text': 'мурод'}, [, {'type': 'mention_name', 'text': '1944742500', 'user_id': 1944742500}, ] [, {'type': 'hashtag', 'text': '#user1944742500'}, ]\nВремя: 24 августа 2021 г. 18:27:48 UTC]",Журнал НИ,channel1212843637


In [51]:
spam.columns

Index(['id', 'type', 'date', 'actor', 'actor_id', 'action', 'title', 'text',
       'from', 'from_id'],
      dtype='object')

In [52]:
for col in ['type', 'actor', 'actor_id', 'action', 'title', 'from', 'from_id']:
    print(col, spam[col].unique())

type ['service' 'message']
actor ['Журнал НИ' nan]
actor_id ['channel1212843637' nan]
action ['create_channel' nan]
title ['Журнал НИ' nan]
from [nan 'Журнал НИ']
from_id [nan 'channel1212843637']


In [53]:
spam = spam['text']
spam.tail(5)

3093                                    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'code', 'text': 'Новое Измайлово'},  [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'},  , {'type': 'code', 'text': 'Tanya'},  [, {'type': 'mention', 'text': '@lunaverrrr'}, ] [, {'type': 'hashtag', 'text': '#user504256170'}, ]\nВремя: 24 августа 2021 г. 10:25:43 UTC]
3094    [🆔 , {'type': 'hashtag', 'text': '#НовыйПользователь'}, \n\n, {'type': 'bold', 'text': 'Группа:'},  , {'type': 'code', 'text': 'Новое Измайлово'},  [, {'type': 'mention', 'text': '@new_izmailovo'}, ] [, {'type': 'hashtag', 'text': '#chat1001077014744'}, ]\n, {'type': 'bold', 'text': 'Пользователь:'},  , {'type': 'code', 'text': 'Александр'},  [, {'type': 'mention_name', 'text': '1993449889', 'user_id': 1993449889}, ] [, {'type': 'hashtag', 'text': '#user1993449889'

In [54]:
spam.iloc[2007]

['🆘 Пользователь наказан командой mute\n7e0afb20-cfa8-41c9-ba41-5113094328fc\n\n',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 '] ',
 {'type': 'italic', 'text': 'ограничил возможность писать сообщения'},
 ' ',
 {'type': 'code', 'text': 'Артур'},
 ' [',
 {'type': 'mention_name', 'text': '1510465722', 'user_id': 1510465722},
 '] [',
 {'type': 'hashtag', 'text': '#user1510465722'},
 '] на 24 часов\n\n\n\n',
 {'type': 'bold', 'text': 'Группа:'},
 ' ',
 {'type': 'code', 'text': 'Новое Измайлово'},
 ' [',
 {'type': 'mention', 'text': '@new_izmailovo'},
 '] [',
 {'type': 'hashtag', 'text': '#chat1001077014744'},
 ']\n',
 {'type': 'bold', 'text': 'Инициатор:'},
 ' ',
 {'type': 'code', 'text': 'Sunlighter'},
 ' [',
 {'type': 'mention', 'text': '@sunlighter'},
 '] [',
 {'type': 'hashtag', 'text': '#user1177305831'},
 ']\n',
 {'type': 'bold', 'text': 'Пользователь:'},
 ' ',
 {'type': 'code',

In [55]:
clean_msg(spam.iloc[2007])

'Добрый день, рады представить Вам тематический чат для Дачников, здесь мы обмениваемся советами по строительству, садоводству и т.д.\n\nhttps://t.me/dacha_pro'

In [56]:
spam = spam.apply(clean_msg)
spam.head(15)

0                                           
1     Канал успешно добавлен в журнал группы
2                                       None
3                                       None
4                                       None
5                                       None
6                                       None
7                                       None
8                                       None
9                                       None
10                                      None
11            Охуенно надежный дом Балашихе😳
12                                      None
13                                      None
14                                      None
Name: text, dtype: object

In [57]:
# извлечение из json текста, удаление строк (сообщений) без текста,
# обновление индекса после удаление строк (индекс теперь по порядку)
spam = spam.apply(clean_msg).dropna().reset_index(drop=True)

In [58]:
spam

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
1                                                                                                                                                                                                                                                            

## Получение объединённого датасета

In [59]:
# Series -> DataFrame
ham_df = ham.to_frame('msg')
# создание нового столбца с категорией спам / не спам
ham_df['spam'] = 0

spam_df = spam.to_frame('msg')
spam_df['spam'] = 1

# конкатенация датафреймов
df = pd.concat([ham_df, spam_df], axis=0, ignore_index=True)
df

,msg,spam
0,Че за возня опять? Возле верного,0
1,Что то похожее на стрельбу было и сейчас три машины полицейских,0
2,А кто-то в курсе что за стрельба?,0
3,"Предупредительный был, видимо , от ментов",0
4,Всё закончилось. Разборки были с поножовщиной,0
...,...,...
18388,"Всем привет, никто не желает учить китайский язык? Я репетиторствую китайским с 2016 года, живу на улице Демин Луг, если есть вопросы и желание пишите )",1
18389,"Уважаемые родители ❗️❗️❗️\nХочу предложить Вам услуги няни👩&zwj;👧&zwj;👦\n\nСобираю группу деток (2-5 детей) для присмотра после садика (с19:00 до 22-23:00) у себя дома. \nВ квартире есть условия для свободной игры, игрушки, горшок, обеспечу вкусным ужином, при необходимости купаю.\n\nСама няня с опытом работы, 🤗могу дать номера телефонов родителей с детками которых ранее присматривала чтоб созвонились узнали обо мне👍\n\n✅Считаю огромным плюсом для родителей то что оплата всего лишь 300 рублей ‼️за вечер 👌",1
18390,"Добрый вечер,могу вам помочь с селективной парфюмерией!\nСама занимаюсь,исключительно оригинал!",1
18391,"Здравствуйте, есть один билет на концерт Сергея Пенкина, завтра в Зелёном Театре, 2000 руб \nНужен сертификат о вакцине или пцр",1


## Сохранение датасета

In [86]:
# сохраняем
df.to_pickle(os.path.join(path, 'df.pkl'))

In [89]:
# загружаем
df = pd.read_pickle(os.path.join(path, 'df.pkl'))
print(df.shape)
df.tail(5)

(18393, 2)


,msg,spam
18388,"Всем привет, никто не желает учить китайский язык? Я репетиторствую китайским с 2016 года, живу на улице Демин Луг, если есть вопросы и желание пишите )",1
18389,"Уважаемые родители ❗️❗️❗️\nХочу предложить Вам услуги няни👩&zwj;👧&zwj;👦\n\nСобираю группу деток (2-5 детей) для присмотра после садика (с19:00 до 22-23:00) у себя дома. \nВ квартире есть условия для свободной игры, игрушки, горшок, обеспечу вкусным ужином, при необходимости купаю.\n\nСама няня с опытом работы, 🤗могу дать номера телефонов родителей с детками которых ранее присматривала чтоб созвонились узнали обо мне👍\n\n✅Считаю огромным плюсом для родителей то что оплата всего лишь 300 рублей ‼️за вечер 👌",1
18390,"Добрый вечер,могу вам помочь с селективной парфюмерией!\nСама занимаюсь,исключительно оригинал!",1
18391,"Здравствуйте, есть один билет на концерт Сергея Пенкина, завтра в Зелёном Театре, 2000 руб \nНужен сертификат о вакцине или пцр",1
18392,⚡️Производим яркие и стильные перегородки в стиле Loft\n \nВиды\n- глухие \n- с распашными дверями\n- с раздвижными дверями\n- книжки\n- гормошки \n\n🔥Мы 20 лет на рынке архитектурно строительных решений из закалённого стекла и конструкционной и нержавеющей стали\n\nПочему выбирают нас? \n\n🔎 Замер бесплатно\n\n🎁 Проект в подарок \n\n⏱ Готовы приступить вчера\n\n🧲 Собственное производство! \n\n✅Работаем строго по договору! \n\n🎯 Гарантия 24 мес! \n\n⚖️ Соблюдаем СНиП и ГОСТ\n\n🏦 У нас есть кредит и рассрочка на:\n-душевые и интерьерные\n перегородки\n-дизайн и ремонт квартир\n-мебель Loft / Hi-tech\n-дизайн и ремонт ванных комнат\nБолее 5 банков-партнёров. \n⚠️Остались вопросы? 👇 \n\nWA http://wa.me/79684198277\n📞 +79684198277,1


In [61]:
df.shape

(18393, 2)

In [82]:
df.spam.value_counts()

0    18047
1      346
Name: spam, dtype: int64

In [84]:
# дисбаланс классов
df.spam.value_counts()[1] / df.shape[0]

0.018811504376665036

# Построение модели

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('spam', 1), 
                                                    df['spam'], random_state=0)
#save test
X_test.to_csv("X_test_spam.csv", index=None)
y_test.to_csv("y_test_spam.csv", index=None)
#save train
X_train.to_csv("X_train_spam.csv", index=None)
y_train.to_csv("y_train_spam.csv", index=None)

In [25]:
X_train

,msg
3856,"Они в один месяц не учли показания (программа сбойнула, не загрузилось с сайта, не туда посмотрели) и у ва..."
40894,"Раз в час примерно, первая в районе 7 утра уходит."
7915,Это же гениально! 😂отличная идея!)))
25371,"Да непонятно, в доброделе отвечают — проведите ОСС и сами залейте)"
40093,"Может одна новая, пробега мало, а вторая загаженная"
...,...
45891,Спасибо 😀
52416,А разве заставляют?
42613,"Не то футбольный праздник, не то свадьба в Чечне."
43567,А чат собаководов есть?


In [26]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [27]:
features = ['msg']
target = 'spam'

In [28]:
#combine
msg = Pipeline([
                ('imputer', TextImputer('msg', '')),
                ('selector', ColumnSelector(key='msg')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('msg', msg)])

In [29]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Wall time: 723 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('msg',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='msg',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='msg')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10))]))])),
                ('classifier', LogisticRegression())])

Посмотрим, как выглядит наш pipeline

In [30]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('msg',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='msg',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='msg')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))]))])),
 ('classifier', LogisticRegression())]

In [31]:
predictions = pipeline.predict_proba(X_test)
#pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

In [32]:
y_test

48502    0
48785    0
40039    0
40399    0
50011    0
        ..
27058    0
16446    0
42301    0
11306    0
31998    0
Name: spam, Length: 13644, dtype: int64

In [33]:
predictions[:, 1][:]

array([0.00166743, 0.00600112, 0.00191033, ..., 0.0022781 , 0.00163441,
       0.00238243])

In [34]:
roc_auc_score(y_score=predictions[:, 1], y_true=y_test)

0.9552632018716577

In [35]:
f1_score(predictions[:, 1].round(), y_test)

0.0

In [36]:
precision_score(predictions[:, 1].round(), y_test)

0.0

In [37]:
recall_score(predictions[:, 1].round(), y_test)

C:\Users\Alex\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [38]:
df_pred = pd.concat([df, pd.Series(predictions[:, 1])], axis=1)
df_pred.columns=["msg", "spam", "prob"]
df_pred

,msg,spam,prob
0,Всем ✋🏻,0,0.001667
1,Это супергруппа. А это значит что админ может редактировать и удалять любые сообщения. Пользователи могут ...,0,0.006001
2,/,0,0.001910
3,"не пугайтесь, немного почистил мусор)",0,0.001192
4,Всем привет),0,0.004088
...,...,...,...
54569,"Добрый день скоро открытия огромного магазина для мастеров и любителей ,для парикмахеров и салонов красоты...",1,NaN
54570,https://t.me/MoscowskyPravdorub/1470?single,1,NaN
54571,Большой выбор ждём вас,1,NaN
54572,Интернет от 300 Мбит/с + бесплатный роутер \nЗвонить с 09:00 до 24:00 без выходных.МГТС/МТС home\nТелефон:...,1,NaN


In [39]:
df_pred.iloc[54570,2], type(df_pred.iloc[54570,2])

(nan, numpy.float64)

In [40]:
pd.set_option('display.max_rows', 200)
df_pred.loc[df_pred['prob'] > 0.1]

,msg,spam,prob
3458,Брат окно,0,0.165447
4849,Воробей всегда подрузамевает))))))))),0,0.143628
6068,Они по проекту района там предполагались,0,0.161846
7398,Желания нет продолжать эту беседу...,0,0.161846
8806,https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku,0,0.161846
9038,Включенные габариты,0,0.161846
9071,"Да, смартфон Android, компьютер Windows",0,0.117427
9257,"Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...",0,0.161846
9632,"И кстати вот тут не за что извиняться мы ж русские , и ""вы"" на Руси было оскорбительно",0,0.166958
9922,ЗАметано),0,0.111680


In [41]:
pd.set_option('display.max_rows', 200)
df_pred.query('(prob > 0.15)').head(200)

,msg,spam,prob
3458,Брат окно,0,0.165447
6068,Они по проекту района там предполагались,0,0.161846
7398,Желания нет продолжать эту беседу...,0,0.161846
8806,https://mir24.tv/news/16293475/v-podmoskovnom-reutove-za-pytki-nad-zhivotnymi-zaderzhali-devushku,0,0.161846
9038,Включенные габариты,0,0.161846
9257,"Подскажите пожалуйста, где у нас в районе можно зеркало на машине починить? Мне кто-то ночью снёс правое з...",0,0.161846
9632,"И кстати вот тут не за что извиняться мы ж русские , и ""вы"" на Руси было оскорбительно",0,0.166958
10149,"Ну и ищите, пойду инвесторов в дивизию предлагать",0,0.161846
10625,"Если брать за понятие ""живёт"" - ставит себе на стол тарелку с супом, то да ЖИВЁТ. А если брать возможности...",0,0.179452
11090,"жена тоже рыба блять, молчит в тряпочку",0,0.161846


Сохраним модель (пайплайн)

In [42]:
with open("spam_classifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)